In [320]:
# import numpy as np
import pandas as pd
%pylab
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import itertools
import networkx as nx
from collections import OrderedDict
from collections import Counter
from networkx.algorithms import approximation as apxa
from sklearn import cluster

rcParams.update({'font.size': 23})
rc("text", usetex = True)
rc("font", family = "serif")
rcParams['figure.subplot.hspace'] = 0.3
rcParams['figure.subplot.wspace'] = 0.1
mpl.rcParams['lines.linewidth'] = 4

sns.set(style='whitegrid', palette='muted',rc={'figure.figsize': (8,6)})
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [81]:
def cleanData(df): 
    df =  df.loc[df.Categories == 'Recetas']
    df = df[df.Title.notnull()]
    
    del df['Content']
    del df['Excerpt']
    del df['URL']
    
    for x in df.columns:
        #eliminate vacum columns
        if df[x].count() < 1:
            del df[x]
        #eliminate _ingredient columns. They have useless information    
#         elif x.startswith('_ingredient'):
        elif x.startswith('_'):
            del df[x]
        #eliminate aterisco columns. They have useless information     
        elif x.endswith('asterisco'):
            del df[x]
        elif x.endswith('condicion'):
            del df[x]

    for x in df.columns:
        #cleaning data of ingredient columns
        if x.startswith('ingrediente') & x.endswith('ingrediente') :
            #Separate strings by '|'.
            ingredients = df[x].str.split(pat = "|",expand=True)
            num = x.split('_') ; num = num[1] #get number of column
            df['Ingrediente_'+ num] = ingredients[0] #create new column with the ingredient

            
            
            #Add another ingredient to the columns if there is a diferent value in one of them 
            if 1 in ingredients:
                condition = ((ingredients[0].notnull()) 
                           & (ingredients[0] != ingredients[1]) 
                           & (ingredients[1].notnull()))
                df.loc[condition, 'Ingrediente_'+ num ] = (ingredients.loc[condition, 0]  
                                                         + ','
                                                         + ingredients.loc[condition, 1]) 
            del df[x]
            
       
#         if x.startswith('paso') & x.endswith('descripcion_del_paso') :    
#     for x in df.columns:
#         #cleaning data of ingredient columns
#         if x.startswith('Ingrediente')       
        
    return df       
# df = cleanData(df)

In [277]:
def replaceValues(df, x):
    df[x] = df[x].fillna(value = ' ')
    df[x] = df[x].str.replace('®' , '' )
    df[x] = df[x].str.replace('<sup>®</sup>' , '' )
    df[x] = df[x].str.replace('<sup></sup>' , '' )
    df[x] = df[x].str.replace('<sup>®</sup>' , '' )
    df[x] = df[x].str.replace('®' , '' )
    df[x] = df[x].str.replace('<strong>' , '' )
    df[x] = df[x].str.replace('</strong>' , '' )
    df[x] = df[x].str.replace('<b>' , '' )
    df[x] = df[x].str.replace('</b>' , '' )
    df[x] = df[x].str.replace('<i>' , '' )
    df[x] = df[x].str.replace('</i>' , '' )
    df[x] = df[x].str.replace(')' , '' )
    df[x] = df[x].str.replace('(' , '' )
    df[x] = df[x].str.replace('?' , '' )
    df[x] = df[x].str.replace('¿' , '' )
    df[x] = df[x].str.replace('¡' , '' )
    df[x] = df[x].str.replace('!' , '' )
    df[x] = df[x].str.replace('/' , '' )

    return df[x]

In [278]:
df = pd.read_csv('Recetas.csv', delimiter = ';')
df[{'Title','Categories'}].loc[df.Categories == 'Recetas'].count()
df = cleanData(df)
 
s = ['Title', 'Tags', 'Total_Ingredientes']

# df['Tags'] = df['Tags'].str.split(pat = "|")
df['Tags'] = df['Tags'].str.replace('|' , ' , ' )

x = 'Ingredientes_ingrediente'
df[x] = replaceValues(df, x)



df['Total_Ingredientes'] = df['Ingredientes_ingrediente']
del df['Ingredientes_ingrediente']

for x in df.columns:
    if x.startswith('Ingre') & (not x.endswith('cantidad')) & (not x.endswith('ingrediente')):
        df[x] = replaceValues(df, x)
        
        df['Total_Ingredientes']  = df['Total_Ingredientes'] + ' , ' + df[x]
        
        # df['Tags'] = df['Tags'].str.split(pat = "|")
        
        del df[x]
        
        
# for x in df.columns:
#      if x.startswith('paso') & x.endswith('descripcion_del_paso') :
#         df[x] = df[x].fillna(value = ' ')
#         df[x] = df[x].str.replace('|' , ' , ' )
#         df[x] = df[x].str.replace('.' , ' , ' )
#         df[x] = df[x].str.split('<b>')
#         df[x] = df[x].
#         s.append(x)

       
# set(s)
df[s]
data = df[s].copy()

In [279]:
data

,Title,Tags,Total_Ingredientes
0,Pollo Cancún,"Aceite de oliva , Cena , Cilantro , Fruta trop...",", DOLE Pineapple Slices piña en rebanadas, s..."
3,Paletas heladas caribeñas de crema,"Fruta tropical , Mango , Miel , Papaya , Postr...",", Jarred DOLE Tropical Fruit fruta tropical ..."
6,Pizza de carnitas y piña,"Cena , Cerdo , Chipotle , Cilantro , Queso",", , , , , , , , , , , ..."
7,Pescado a la veracruzana,"Aceite de oliva , Arroz , Cena , Cilantro , Ja...",", , , , , , , , , , , ..."
10,Lomo con chimichurri a la parrilla,"Aceite de oliva , Cena , Cilantro , Lomo , Res...",", Hojas frescas de perejil italiano , Aceite..."
11,Fajitas de pollo y frijoles negros,"Cena , Frijoles , Pollo , Queso , Salsa , Tort...",", , , , , , , , , , , ..."
14,Rollo de galletas brazo gitano,"Fruta tropical , Postre",", Galletas redondas de vainilla , Jugo DOLE ..."
17,Guacamole con fruta tropical,"Aguacate , Guacamole","Aguacates o paltas machacados, sin semilla ni ..."
18,Biónico,"Frutas , Miel , Postre",", DOLE Pineapple Chunks piña en trozos, sin ..."
21,Ceviche de camarón,"Ceviche , Cilantro , Mariscos",", Camarones pequeños, cocidos , DOLE Pineapp..."


In [280]:

for x in data.columns:
    if x != 'Title':
        data['Title']  = data['Title'] + ' , ' + data[x]
        del data[x]
        


In [281]:
data.head()

,Title
0,"Pollo Cancún , Aceite de oliva , Cena , Cilant..."
3,"Paletas heladas caribeñas de crema , Fruta tro..."
6,"Pizza de carnitas y piña , Cena , Cerdo , Chip..."
7,"Pescado a la veracruzana , Aceite de oliva , A..."
10,"Lomo con chimichurri a la parrilla , Aceite de..."


In [282]:
[]

[]

In [391]:
words = array(data['Title'])
Recetas = []
# These words are going to be eliminated from the list
delated = [ ''
           ,'de'
           ,'y'
           ,'la'
           ,'el'
           ,'en'
           ,'una'
           ,'con'
           ,'los'
           ,'un'
           ,'hasta'
           ,'para'
           ,'a'
           ,'que'
           ,'o'
           ,'las'
           ,'los'
           ,'por'
           ,'al'
           ,'del'
           ,'esté'
           ,'sobre'
           ,'1'
           ,'2'
           ,'3'
           ,'se'
           ,'durante'
           ,'sin'
           ,'coloca'
           ,'minutos']
for w in words:
    st = w.split(',')
    l = []
    for s in st:
        x = s.split(' ')
        l = l + x
    l = [x.lower().strip() for x in l] #strip each word
    l = list(filter(lambda a: a not in delated, l)) # clean misleading words
    l = list(OrderedDict.fromkeys(l))  #Eliminate repeated words in a row
    Recetas.append(l)       #fd

for r in Recetas:
    for j in range(len(r)): 
        if 'dole' in r[j]:
            r[j] = 'dole'

# eliminating next  part whe attempting the final model           
Recetas = Recetas[:]



Words = list(itertools.chain.from_iterable(Recetas))
Word_labels = list(OrderedDict.fromkeys(Words))
# W = pd.DataFrame(Words)


# W[0].value_counts()

# Word_labels
c = Counter(Words)
h = c.most_common(60)
h = array(h)
Word_labels = h[:,0]

In [392]:
# Word_labels
# Recetas[0]
Word_labels

array(['dole', 'piña', 'jugo', 'pineapple', 'azúcar', 'sal', 'trozos',
       'piñas', 'postre', 'limón', 'harina', 'bebida', 'leche', 'canela',
       'dulce', 'tropical', 'salsa', 'mantequilla', 'rodajas', 'agua',
       'coco', 'vainilla', 'trituradas', 'polvo', 'aceite', 'cena',
       'fruta', 'crema', 'queso', 'fácil', 'escurridas', 'cilantro',
       'juice', 'condensada', 'naranja', 'delicioso', 'méxico',
       'extracto', 'común', 'lima', 'molida', 'crushed', 'cerdo',
       'cebolla', 'ajo', 'verde', 'maíz', 'desayuno', 'huevos', 'blanco',
       'alcohólica', 'pollo', 'finamente', 'tidbits', 'pimienta',
       'chunks', 'ron', 'cóctel', 'sabor', 'pastel'], dtype='<U11')

In [399]:
G = nx.Graph()
G.add_nodes_from(array(Word_labels))
Recetas_copy = Recetas.copy()
elist = []
nodeSize = zeros(len(Word_labels))

alpha = 1/len(Recetas)
# 'piña' in m
for i in range(len(Word_labels)):
    w = Word_labels[i]
    for j in arange(i+1 , len(Word_labels)):
        w2 = Word_labels[j]
#         print(w,w2)
        count = 0
        for R in Recetas:
            if (w in R) & (w2 in R):
                count = count +1
                nodeSize[i] = nodeSize[i] + 1
                nodeSize[j] = nodeSize[j] + 1
#                 print(count)
        if count >2:
            elist.append((w,w2,count*alpha))
G.add_weighted_edges_from(elist)
    
    
#     for R in Recetas_copy:
#         if (w in R):
#             for w2 in R:
#                 if (w2 in Word_labels):
#                     G.add_edge(w, w2)
#         found = False
#         while (found = False)
        
#             if 
eds = array(elist)
    
G2 = nx.ego_graph(G, 'dole')
nodes = list(G2.nodes())
nodeSizes = zeros(len(nodes))
for i in range(len(nodes)):
    x = nodes[i]
    nodeSizes[i] = len([n for n in G2.neighbors(x)])
# nodeSizes
mini = min(eds[:,2])
maxi = max(eds[:,2])
# print(mini,maxi)
# eds[:,2]
# graphs = list(nx.connected_component_subgraphs(G))
# G2 = graphs[0]

array([59., 59., 58., 52., 48., 49., 42., 40., 30., 36., 37., 22., 28.,
       37., 33., 20., 18., 33., 20., 29., 24., 32., 28., 27., 23., 21.,
       23., 29., 28., 17., 22., 16., 13., 20., 14., 11., 17., 29., 22.,
       13., 24., 25.,  9., 17., 18., 16., 12., 15., 21., 15., 16.,  7.,
       14., 13., 14.,  6.,  6.,  9., 14., 18.])

0.03125 0.7083333333333333


In [403]:

weights

(0.7083333333333333,
 0.5625,
 0.375,
 0.3333333333333333,
 0.29166666666666663,
 0.29166666666666663,
 0.22916666666666666,
 0.16666666666666666,
 0.19791666666666666,
 0.1875,
 0.17708333333333331,
 0.14583333333333331,
 0.16666666666666666,
 0.15625,
 0.14583333333333331,
 0.125,
 0.14583333333333331,
 0.14583333333333331,
 0.15625,
 0.14583333333333331,
 0.13541666666666666,
 0.14583333333333331,
 0.13541666666666666,
 0.11458333333333333,
 0.08333333333333333,
 0.125,
 0.13541666666666666,
 0.10416666666666666,
 0.11458333333333333,
 0.125,
 0.08333333333333333,
 0.11458333333333333,
 0.09375,
 0.10416666666666666,
 0.08333333333333333,
 0.09375,
 0.10416666666666666,
 0.09375,
 0.10416666666666666,
 0.09375,
 0.10416666666666666,
 0.05208333333333333,
 0.09375,
 0.08333333333333333,
 0.09375,
 0.08333333333333333,
 0.09375,
 0.08333333333333333,
 0.08333333333333333,
 0.09375,
 0.05208333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.0833333333333

In [404]:
print(G2.number_of_nodes())
nx.draw_networkx(G2
                 , node_size = nodeSize
                 , with_labels = True
                 , node_color= 'r'
                 , edgelist = edges
                 , width= 1.0
                 , edge_color = weights
                 , edge_cmap = cm.Blues
                 , edge_vmin = 0 
                 , edge_vmax = 0.1)
# print(nx.triangles(G,'piña'))

60


/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [ ]:
l = [1,2,4,1,3,4,6,7,8,1,4]
list(filter(lambda a: a not in [1 , 4, 8], l)) 

In [111]:
dolead = G.adjacency()
# G.add_node(1)
# G.add_node('b')
G.subgraph(dolead['dole'])

TypeError: 'dict_itemiterator' object is not subscriptable

In [101]:
k_components = apxa.k_components(G)

In [103]:
k_components[1]

[{'3',
  '<i>light<i>',
  'aceite',
  'aguacate',
  'arroz',
  'arándanos',
  'baja',
  'brazo',
  'calorías',
  'cancún',
  'caribeñas',
  'carnitas',
  'cebolla',
  'cena',
  'cerdo',
  'chimichurri',
  'chipotle',
  'chunks',
  'cilantro',
  'clavo',
  'coco',
  'comino',
  'condensada',
  'cortados',
  'crema',
  'deshuesadas',
  'desmenuzado',
  'dole',
  'envasada',
  'fajitas',
  'finamente',
  'frescas',
  'fresco',
  'frijoles',
  'fruit',
  'fruta',
  'galletas',
  'gitano',
  'guacamole',
  'heladas',
  'hojas',
  'italiano',
  'jalapeño',
  'jarred',
  'jugo',
  'juice',
  'leche',
  'limón',
  'lomo',
  'mango',
  'miel',
  'mixed',
  'moldes',
  'molido',
  'negros',
  'nueces',
  'néctar',
  'oliva',
  'orégano',
  'oz',
  'paletas',
  'papaya',
  'parrilla',
  'pecanas',
  'pelado',
  'pequeños',
  'perejil',
  'pescado',
  'picada',
  'piel',
  'pimienta',
  'pineapple',
  'pizza',
  'piña',
  'plástico',
  'plátano',
  'pollo',
  'postre',
  'queso',
  'rebanadas',
  

In [96]:
G = nx.petersen_graph()
k_components = apxa.k_components(G)

In [97]:
k_components

defaultdict(list,
            {1: [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}],
             2: [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}],
             3: [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}]})

In [98]:
nx.draw_networkx(G)

/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [ ]:
df['Ingredientes_ingrediente'].str.split(pat = "|",expand = True)

In [ ]:
display_all(df.describe(include='all').T)

In [ ]:
df['tags'] = df['Tags'].str.split(pat = "|")

In [ ]:
df[{'Title','tags'}]

In [ ]:
df.head()

In [ ]:
df['ingredientes_11_asterisco'].value_counts()

In [ ]:
s = {'Ingredientes_ingrediente', 'Ingredientes_cantidad' , 'Ingredientes_nombre_de_condicion'}
df[s]

In [ ]:
x = 'ingredientes_9_ingrediente'
ingredients = df[x].str.split(pat = "|",expand=True)
# ingredients[(ingredients[0].notnull()) & (ingredients[0] != ingredients[1]) & (ingredients[1].notnull()) ]
# ingredients = df.ingredientes_17_ingrediente.notnull()
# ingredients[df.ingredientes_17_ingrediente.notnull()]
# 1 in ingredients
num = x.split('_')
num = num[1]
print(num)
df['Ingrediente_'+ num] = ingredients[0] 

if 1 in ingredients:
                print('entered')
                condition = ((ingredients[0].notnull()) 
                           & (ingredients[0] != ingredients[1]) 
                           & (ingredients[1].notnull()))
                    
                
#                 df['Ingrediente_'+ num] = ingredients[0] +','+ ext[1]

df.loc[condition, 'Ingrediente_'+ num ] = (ingredients.loc[condition, 0]  + ','+ ingredients.loc[condition, 1] ) 
condition = ((ingredients[0].notnull()) 
                           & (ingredients[0] != ingredients[1]) 
                           & (ingredients[1].notnull()))

# df['Ingrediente_'+ num]
# ext
df.loc[condition,'Ingrediente_'+ num]

In [ ]:
display_all(df.describe(include='all').T)

In [ ]:
df['Nombre del Autor'].value_counts()

In [ ]:
# for x in df.columns:
#     if df[x].count() < 10:
#         del df[x]
df.head()

In [ ]:
df['Categories'].value_counts()

In [ ]:
df['ingredientes_0_cantidad'].value_counts()

In [ ]:
df['_ingredientes_0_cantidad'].value_counts()

In [ ]:
df['Content'].value_counts()